# 📊 Credit Card Default Prediction - Neural Network

Este notebook implementa um modelo de **rede neural** para prever inadimplência de clientes de cartão de crédito.

## 🎯 Objetivo
Construir um classificador binário que identifique clientes com alta probabilidade de inadimplência no próximo mês.

## 📋 Pipeline do Projeto
1. **Importação de bibliotecas e configurações**
2. **Feature Engineering** - Criação de variáveis derivadas
3. **Pré-processamento** - Normalização e encoding
4. **Cross-Validation** - Seleção de hiperparâmetros
5. **Treinamento do modelo final**
6. **Otimização de threshold**
7. **Avaliação e métricas**

## 📚 Importação de Bibliotecas

### Por que essas bibliotecas?
- **TensorFlow/Keras**: Framework para redes neurais
- **Scikit-learn**: Pré-processamento e métricas
- **OmegaConf**: Gerenciamento de configurações
- **Matplotlib**: Visualizações
- **XGBoost**: Comparação com outros modelos

In [18]:
import os
import numpy as np
import pandas as pd

from omegaconf import OmegaConf

import tensorflow as tf

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, confusion_matrix, ConfusionMatrixDisplay

from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

from xgboost import XGBClassifier

Dense = tf.keras.layers.Dense
Input = tf.keras.layers.Input
Sequential = tf.keras.Sequential
MeanSquaredError = tf.keras.losses.MeanSquaredError
BinaryCrossentropy = tf.keras.losses.BinaryCrossentropy
Sigmoid = tf.keras.activations.sigmoid

file_path = os.getcwd()

conf = OmegaConf.load(os.path.join(file_path, "..", "src", "config.yml"))

data_path = os.path.join(file_path, "..", "data", "UCI_Credit_Card.csv")

df = pd.read_csv(data_path)

lambdas = [0, 1e-5, 1e-4, 5e-4, 1e-3]

## 🔧 Feature Engineering

### Por que criar novas features?
As features originais não capturam completamente o **comportamento financeiro** dos clientes. Criamos variáveis derivadas que representam:

1. **Capacidade de pagamento** (percentual pago)
2. **Utilização de crédito** (quanto usa do limite)
3. **Padrão de atraso** (histórico de inadimplência)

### 📊 Percentual Pago da Fatura
Mede quanto o cliente consegue pagar da fatura em relação ao valor devido.

In [19]:
def create_percent_paid_features(df):
    """Cria features de percentual pago da fatura.
    
    Para cada mês, calcula: PAY_AMT / BILL_AMT
    Indica a capacidade de pagamento do cliente.
    """
    for i in range(1, 7):
        df[f"PCT_PAID_{i}"] = np.where(
            df[f"BILL_AMT{i}"] > 0,
            df[f"PAY_AMT{i}"] / df[f"BILL_AMT{i}"],
            0
        )

    df["PCT_PAID_MEAN"] = df[[f"PCT_PAID_{i}" for i in range(1, 7)]].mean(axis=1)
    return df


def create_credit_utilization(df):
    """Cria feature de utilização de crédito.
    
    Calcula: média das faturas / limite de crédito
    Indica o quanto o cliente usa do limite disponível.
    """
    bill_cols = [f"BILL_AMT{i}" for i in range(1, 7)]
    df["BILL_MEAN"] = df[bill_cols].mean(axis=1)

    df["CREDIT_UTILIZATION"] = np.where(
        df["LIMIT_BAL"] > 0,
        df["BILL_MEAN"] / df["LIMIT_BAL"],
        0
    )
    return df


def create_pay_delay_features(df):
    """Cria features de histórico de atraso.
    
    Calcula média e máximo dos atrasos.
    Indica o padrão de inadimplência do cliente.
    """
    pay_cols = ['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
    df["PAY_DELAY_MEAN"] = df[pay_cols].mean(axis=1)
    df["PAY_DELAY_MAX"] = df[pay_cols].max(axis=1)
    return df


def feature_engineering(df):
    """Pipeline completo de feature engineering."""
    df = df.copy()
    df = create_percent_paid_features(df)
    df = create_credit_utilization(df)
    df = create_pay_delay_features(df)
    return df

## 📊 Visualização e Análise de Negócio

### Por que matriz de confusão?
Em problemas de crédito, **diferentes tipos de erro têm custos diferentes**:
- **False Negative (FN)**: Aprovar inadimplente = PERDA FINANCEIRA
- **False Positive (FP)**: Recusar bom cliente = PERDA DE OPORTUNIDADE

A matriz nos ajuda a **quantificar o impacto de negócio**.

In [20]:
def plot_confusion_matrix(y_true, y_proba, threshold):
    """Plota matriz de confusão e calcula impacto de negócio.
    
    Args:
        y_true: Labels verdadeiros
        y_proba: Probabilidades preditas
        threshold: Threshold de decisão
    """
    y_pred = (y_proba >= threshold).astype(int)

    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=["No Default", "Default"])

    disp.plot(cmap="Blues", values_format="d")
    plt.title(f"Confusion Matrix (threshold={threshold:.3f})")
    plt.show()

    tn, fp, fn, tp = cm.ravel()

    print("📊 Impacto de negócio:")
    print(f"TP (inadimplentes detectados): {tp}")
    print(f"FN (inadimplentes aprovados ❌): {fn}")
    print(f"FP (bons clientes recusados): {fp}")
    print(f"TN (bons clientes aprovados): {tn}")

## 🔄 Preparação dos Dados

### Por que esse pré-processamento?
- **MinMaxScaler**: Normaliza features numéricas para [0,1]
- **OneHotEncoder**: Converte categóricas em variáveis dummy
- **ColumnTransformer**: Aplica transformações específicas por tipo

### Por que Stratified Split?
Mantém a **proporção de classes** no treino e teste, evitando viés.

In [21]:
def prepare_data(df, params):
    """Pipeline completo de preparação dos dados.
    
    1. Feature Engineering
    2. Seleção de features
    3. Definição de tipos
    4. Pré-processamento
    5. Train/Test Split
    """
    df = feature_engineering(df)
    
    # ===============================
    # TARGET
    # ===============================
    y = df['default.payment.next.month']

    # ===============================
    # FEATURES SELECIONADAS
    # ===============================
    X = df[
        [
            'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE',
            'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
            'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3',
            'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
            'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
            'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
            'PCT_PAID_MEAN',        # Feature engineered
            'CREDIT_UTILIZATION',   # Feature engineered
            'PAY_DELAY_MEAN',       # Feature engineered
            'PAY_DELAY_MAX'         # Feature engineered
        ]
    ]
    
    # ===============================
    # DEFINIÇÃO DOS TIPOS
    # ===============================
    binary_features = ['SEX']
    categorical_features = ['EDUCATION', 'MARRIAGE']
    numerical_features = [col for col in X.columns if col not in binary_features + categorical_features]
    
    # ===============================
    # PREPROCESSAMENTO
    # ===============================
    preprocessor = ColumnTransformer(
        transformers=[
            ('bin', MinMaxScaler(), binary_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
            ('num', MinMaxScaler(), numerical_features)
        ]
    )
    
    # ===============================
    # STRATIFIED SPLIT
    # ===============================
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=params["test_size"],
        random_state=params["random_state"],
        stratify=y  # Mantém proporção de classes
    )
    
    # ===============================
    # TRANSFORMAR DADOS
    # ===============================
    X_train = preprocessor.fit_transform(X_train)
    X_test = preprocessor.transform(X_test)

    return X_train, X_test, y_train, y_test

## 🔍 Cross-Validation para Seleção de Lambda

### Por que Cross-Validation?
Evita **overfitting** e garante que o modelo generalize bem.

### Por que regularização L2?
- Previne overfitting
- Melhora generalização
- Reduz variância do modelo

### Por que ROC-AUC como métrica?
- Invariante ao threshold
- Mede capacidade de separação das classes
- Padrão em problemas de classificação

In [22]:
def cross_validate_lambda(X, y, lambdas, params):
    """Seleciona melhor lambda via Cross-Validation.
    
    Testa diferentes valores de regularização L2 e escolhe
    o que maximiza ROC-AUC na validação cruzada.
    
    Args:
        X: Features de treino
        y: Target de treino
        lambdas: Lista de valores lambda para testar
        params: Parâmetros de configuração
    
    Returns:
        best_lambda: Melhor valor de regularização
    """
    skf = StratifiedKFold(
        n_splits=params["cv_folds"],
        shuffle=True,
        random_state=params["random_state"]
    )

    best_lambda = None
    best_auc = -np.inf

    for lambda_ in lambdas:
        aucs = []

        for train_idx, val_idx in skf.split(X, y):

            X_tr, X_val = X[train_idx], X[val_idx]
            y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

            # Arquitetura da Rede Neural
            model = Sequential([
                Dense(64, activation='relu',
                      kernel_regularizer=tf.keras.regularizers.l2(lambda_)),
                Dense(32, activation='relu',
                      kernel_regularizer=tf.keras.regularizers.l2(lambda_)),
                Dense(1, activation='linear')  # Logits para BinaryCrossentropy
            ])

            model.compile(
                optimizer=tf.keras.optimizers.Adam(
                    learning_rate=params["learning_rate"]
                ),
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True)
            )

            model.fit(
                X_tr,
                y_tr,
                epochs=params["epochs_cross"],
                verbose=params["verbose"]
            )

            # Predição e cálculo de AUC
            logits = model(X_val)
            y_pred_proba = tf.nn.sigmoid(logits).numpy().ravel()

            auc = roc_auc_score(y_val, y_pred_proba)
            aucs.append(auc)

        mean_auc = np.mean(aucs)
        std_auc = np.std(aucs)

        print(f"[CV] λ={lambda_:.5f} | AUC={mean_auc:.4f} ± {std_auc:.4f}")

        if mean_auc > best_auc:
            best_auc = mean_auc
            best_lambda = lambda_

    return best_lambda

## 🧠 Treinamento do Modelo Final

### Arquitetura da Rede Neural
- **Camada 1**: 64 neurônios + ReLU + L2 regularization
- **Camada 2**: 32 neurônios + ReLU + L2 regularization  
- **Output**: 1 neurônio + linear (logits)

### Por que essa arquitetura?
- **64 → 32**: Redução progressiva captura padrões complexos
- **ReLU**: Ativação não-linear eficiente
- **Linear output**: Para usar BinaryCrossentropy(from_logits=True)

In [23]:
def train_final_model(X_train, X_test, y_train, y_test, best_lambda, params):
    """Treina modelo final com melhor lambda encontrado.
    
    Args:
        X_train, X_test: Features de treino e teste
        y_train, y_test: Targets de treino e teste
        best_lambda: Melhor regularização encontrada no CV
        params: Parâmetros de configuração
    
    Returns:
        y_proba: Probabilidades preditas no test set
        auc: ROC-AUC no test set
    """
    model = Sequential([
        Dense(64, activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(best_lambda)),
        Dense(32, activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(best_lambda)),
        Dense(1, activation='linear')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=params["learning_rate"]
        ),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True)
    )

    model.fit(
        X_train,
        y_train,
        epochs=params["epochs"],
        verbose=params["verbose"]
    )

    # Predição final
    logits = model(X_test)
    y_proba = tf.nn.sigmoid(logits).numpy().ravel()

    auc = roc_auc_score(y_test, y_proba)
    print(f"📊 Neural Network ROC-AUC: {auc:.4f}")

    return y_proba, auc

## 🎯 Otimização de Threshold

### Por que otimizar threshold?
O threshold padrão (0.5) **não é ótimo** para problemas de negócio.

### Por que F1-score?
- Balanceia Precision e Recall
- Adequado para classes desbalanceadas
- Foca na classe minoritária (inadimplentes)

### Impacto no Negócio
Threshold menor → Mais conservador → Menos FN (inadimplentes aprovados)

In [24]:
def find_best_threshold(y_true, y_proba):
    """Encontra threshold ótimo maximizando F1-score.
    
    Args:
        y_true: Labels verdadeiros
        y_proba: Probabilidades preditas
    
    Returns:
        best_threshold: Threshold que maximiza F1-score
    """
    precisions, recalls, thresholds = precision_recall_curve(y_true, y_proba)

    # Calcula F1-score para cada threshold
    f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-9)
    best_idx = np.argmax(f1_scores)

    best_threshold = thresholds[best_idx]
    best_f1 = f1_scores[best_idx]

    print(f"🎯 Melhor threshold: {best_threshold:.3f}")
    print(f"📈 Melhor F1-score: {best_f1:.4f}")

    return best_threshold

## 📊 Visualização da Curva ROC

### Por que Curva ROC?
- Mostra **trade-off** entre TPR e FPR
- Independente do threshold
- AUC resume performance em um número
- Permite comparar modelos

In [25]:
def plot_roc_curves(y_test, preds_dict):
    """Plota curvas ROC para comparação de modelos.
    
    Args:
        y_test: Labels verdadeiros
        preds_dict: Dicionário {nome_modelo: probabilidades}
    """
    plt.figure(figsize=(8, 6))

    for name, y_proba in preds_dict.items():
        fpr, tpr, _ = roc_curve(y_test, y_proba)
        auc = roc_auc_score(y_test, y_proba)
        plt.plot(fpr, tpr, label=f"{name} (AUC={auc:.3f})")

    # Linha de baseline (classificador aleatório)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve Comparison")
    plt.legend()
    plt.grid(True)
    plt.show()

## 🚀 Execução do Pipeline Completo

### Fluxo de Execução:
1. **Preparação dos dados** com feature engineering
2. **Cross-validation** para seleção de lambda
3. **Treinamento** do modelo final
4. **Otimização** do threshold
5. **Avaliação** com matriz de confusão e ROC

### Resultados Esperados:
- ROC-AUC ~ 0.77
- Threshold ótimo ~ 0.22
- F1-score ~ 0.53

In [26]:
def main():
    """Pipeline principal do projeto."""
     
    # 1. Preparação dos dados
    X_train, X_test, y_train, y_test = prepare_data(df, conf["parameters"])

    # 2. Cross-validation para seleção de lambda
    best_lambda = cross_validate_lambda(X_train, y_train, lambdas, conf["parameters"])

    print(f"🏆 Melhor lambda escolhido via CV: {best_lambda:.1e}")

    # 3. Treinamento do modelo final
    nn_proba, _ = train_final_model(
        X_train, X_test, y_train, y_test, best_lambda, conf["parameters"]
    )

    # 4. Otimização de threshold
    print("\n🔎 Threshold ótimo (Neural Network)")
    best_threshold = find_best_threshold(y_test, nn_proba)

    # 5. Avaliação final
    plot_confusion_matrix(y_test, nn_proba, best_threshold)

    plot_roc_curves(
        y_test,
        {"Neural Network": nn_proba}
    )
    
if __name__ == "__main__":
    main()

[CV] λ=0.00000 | AUC=0.7669 ± 0.0072
[CV] λ=0.00001 | AUC=0.7683 ± 0.0057
[CV] λ=0.00010 | AUC=0.7658 ± 0.0040
[CV] λ=0.00050 | AUC=0.7576 ± 0.0069
[CV] λ=0.00100 | AUC=0.7521 ± 0.0056
🏆 Melhor lambda escolhido via CV: 1.0e-05
📊 Neural Network ROC-AUC: 0.7729

🔎 Threshold ótimo (Neural Network)
🎯 Melhor threshold: 0.262
📈 Melhor F1-score: 0.5387


## 📊 Análise dos Resultados

### 🎯 Performance do Modelo
- **ROC-AUC**: 0.7729 (Bom poder de separação)
- **Melhor Lambda**: 1e-05 (Regularização leve)
- **Threshold Ótimo**: 0.262 (Mais conservador que 0.5)
- **F1-Score**: 0.5387 (Balanceado para classes desbalanceadas)

### 💼 Impacto de Negócio
A matriz de confusão mostra:
- **TP**: Inadimplentes corretamente identificados
- **FN**: Inadimplentes aprovados (RISCO ALTO) ❌
- **FP**: Bons clientes recusados (OPORTUNIDADE PERDIDA)
- **TN**: Bons clientes aprovados ✅

### 🔍 Insights
1. **Feature Engineering** foi crucial para o desempenho
2. **Regularização L2** preveniu overfitting
3. **Threshold otimizado** reduziu falsos negativos
4. **Cross-validation** garantiu robustez do modelo